In [1]:
import glimpse
from glimpse.imports import datetime,np,os
import glob
from PIL import Image
from PIL.ExifTags import TAGS
import numpy as np
import matplotlib.pyplot as plt
import os

# LINK TO DROPBOX
https://www.dropbox.com/sh/iqhd68g9shvroic/AABadr95uv0w0jSNry6zd5c-a?dl=0

First, we will define the camera model. We can get the intrinsic parameters using image meta-data, and the extrinsic parameters from surveyed gps data for the camera's location and an initial guess at the camera's view orientation that we can refine later.

In [ ]:
'''
exifinfo = {}
img_path = '/home/dunbar/Research/wolverine/data/cam_cliff/images/cam_cliff_0497.JPG'
image = Image.open(img_path)
exif_info = image._getexif()
for (tag,value) in exif_info.items():
    tagname = TAGS.get(tag,tag)
    exifinfo[tagname] = value
'''

For the purposes of GLIMPSE, we will need the following from the dictionary defined above: Image Dimensions (pixels), Focal Length (mm), Sensor Dimensions (mm), and Date (datetime format).
We can typically get all but the Sensor Dimensions from the images themselves. In this case I had to look it up.

In [ ]:
'''
image_focal = (exifinfo['FocalLength'][0]/exifinfo['FocalLength'][1])
imagewidth = int(exifinfo['ExifImageWidth'])
imageheight = int(exifinfo['ExifImageHeight'])
image_date = exifinfo['DateTime']
'''
sensorx,sensory = 35.9,24

In [2]:
params= dict(
  sensorsz=(35.9, 24),
  xyz=(393506.713, 6695855.64, 961.3370),
  viewdir=(6.55401513e+01, -1.95787507e+01,  7.90589866e+00),
  p=(1.42603100e-01,4.12163305e-01),
  k=(-4.78577912e-01,  1.08772049e+00, -1.27452460e+00,6.78479918e-01,  2.39852156e-02, -1.229e-01),
  imgsz=(7360, 4912),
  fmm=28
)

In [3]:
cliff_camera = glimpse.Camera(**params)

In [5]:
cliff_camera.as_dict()

{'xyz': [393506.713, 6695855.64, 961.337],
 'viewdir': [65.5401513, -19.5787507, 7.90589866],
 'imgsz': [7360.0, 4912.0],
 'f': [5740.389972144847, 5730.666666666667],
 'c': [0.0, 0.0],
 'k': [-0.478577912,
  1.08772049,
  -1.2745246,
  0.678479918,
  0.0239852156,
  -0.1229],
 'p': [0.1426031, 0.412163305]}

Now that we have our camera model defined, we can "fine tune" it's parameters using Glimpse's optimize module. For this, we need real world coordinate to image coordinate coorespondances in respective arrays.

In [ ]:
cliff_imgpts = np.array([[5193, 549],[3101, 642],[6153.0, 2297.0]])
cliff_worldpts = np.array([[408245.86,6695847.03,1560 ],[416067.22,6707259.97,988],[394569.509, 6695550.678, 621.075]])

cliff_points = glimpse.optimize.Points(cliff_camera,cliff_imgpts,cliff_worldpts)
cliff_points.plot()

In [ ]:
cliffparams = glimpse.optimize.Cameras([cliff_camera],[cliff_points],dict(viewdir=True,p=True,k=True))
cliffparams = cliffparams.fit()

In [ ]:
cliffparams

In [ ]:
params= dict(
  sensorsz=(35.9, 24),
  xyz=(393506.713, 6695855.64, 961.3370),
  viewdir=(6.55401513e+01, -1.95787507e+01,  7.90589866e+00),
  p=(1.42603100e-01,4.12163305e-01),
  k=(-4.78577912e-01,  1.08772049e+00, -1.27452460e+00,6.78479918e-01,  2.39852156e-02, -1.229e-01),
  imgsz=(7360, 4912),
  fmm=28
)

In [ ]:
cliff_camera2 = glimpse.Camera(**params)

In [ ]:
cliff_camera2.as_dict()

Now that there is  a reasonably defined camera model, we can create an observer object which holds the images on which GLIMPSE will perform feature tracking. 

In [ ]:
img_path = '/home/dunbar/Research/wolverine/data/cam_cliff/images'
images = glob.glob(os.path.join(img_path,'*.JPG'),recursive=True)
images = [glimpse.Image(path=imagepath,cam=cliff_camera.copy()) for imagepath in images]
images.sort(key= lambda img: img.datetime ) # sort by datetime
images = images[::4]

In [ ]:
temp = images[0].cam.as_dict()

In [ ]:
print(temp)

In [ ]:
def save_observercams(observer,directory,print_path=False):
    if not os.path.isdir(directory):
        raise Exception("Directory {} Not Found".format(directory))
    for images in observer.images:
        filename = images.path.split("/")[-1]
        filename = filename.split(".")[0]
        path = os.path.join(directory,filename)
        path += ".JSON"
        if print_path: print("Path: {}\n".format(path))
        try:
            images.cam.write(path)
        except:
            print("Image {} Has Undefined Camera".format(image.path))


In [ ]:
cliffobserver = glimpse.Observer(images)

In [ ]:
cliffdir = '/home/dunbar/Research/wolverine/data/cam_cliff/images_json'

In [ ]:
save_observercams(cliffobserver,cliffdir)

With our observer in good shape, we can now proceed to prepare the DEM

In [ ]:
dem_path = '/home/dunbar/Research/wolverine/data/dem/wolverine_skyline32606.tif'
dem = glimpse.Raster.read(dem_path,d=4)
dem.crop(zlim=(0,np.inf))
dem.fill_crevasses(mask=~np.isnan(dem.Z), fill=True)
dem.fill_circle(cliffobserver.xyz,radius=100)

In [ ]:
viewshed = dem.copy()
viewshed.Z = np.ones(dem.shape, dtype=bool)
viewshed.Z &= dem.viewshed(cliffobserver.xyz)

In [ ]:
import itertools
xy0= np.array((394368,6696220))
xy = xy0 + np.vstack([xy for xy in
    itertools.product(range(-200, 200, 50), range(-200, 200, 50))])

motion_model = []
time_unit = datetime.timedelta(days=.5)

for i in range(xy.shape[0]):
    motion_model.append(glimpse.CartesianMotionModel(
        xy[i,:], time_unit=time_unit, dem=dem, dem_sigma=3, n=5000, xy_sigma=(2, 2),
        vxyz_sigma=(5, 5, 0.2), axyz_sigma=(2, 2, 0.2)))

In [ ]:
tracker = glimpse.Tracker(observers=[cliffobserver], viewshed=viewshed)

In [ ]:
tracks = tracker.track(motion_models=motion_model, tile_size=(10, 10),parallel=26)

In [ ]:
tracks.plot_xy(start=dict(color='green'), mean=dict(color='red'), sigma=dict(alpha=0.25))